# Load libraries

In [189]:
import numpy as np
import re
import pandas as pd
import os
from datetime import datetime

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Clone repo from GitHub

Link: https://git-scm.com/docs/git-clone <br>
<b>Note:</b> In case too-long file path issue occurs in Windows, set <code>git config --system core.longpaths true</code>

In [34]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [35]:
remote_link = "https://github.com/apache/camel"
local_link = "C:\\Secure Software\\camel"
# Uncomment to clone
# Repo.clone_from(remote_link, local_link, progress=Progress())

In [308]:
repo = Repo(local_link)
fixing_commit = "af3f54de35a90a5a49a4af4622e8bd1011bf5ec"
affected_file = "components-starter/camel-gson-starter/src/main/java/org/apache/camel/component/gson/springboot/GsonDataFormatConfiguration.java"

# A) Message and Title of the Fixing Commit
Link: https://git-scm.com/docs/git-show
You can explore different parameters.

In [328]:
show_data = repo.git.show("-s", fixing_commit).splitlines()
show_data.reverse()
print("Message and title of the commit:")
print(show_data[0])

Message and title of the commit:
    CAMEL-10567: Camel-Jackson: Add an option to allow the UnmarshallType header use. Regenerated code


# B) Total Files Affected

In [151]:
total_files_affected = repo.git.show("--stat",fixing_commit).splitlines()
print("Files Afftected:")
total_files_affected = total_files_affected[6:]
for line in total_files_affected:
    print(line)

Files Afftected:
 .../gson/springboot/GsonDataFormatConfiguration.java       | 14 ++++++++++++++
 .../jackson/springboot/JacksonDataFormatConfiguration.java | 14 ++++++++++++++
 .../johnzon/springboot/JohnzonDataFormatConfiguration.java | 14 ++++++++++++++
 .../xstream/springboot/JsonDataFormatConfiguration.java    | 14 ++++++++++++++
 .../camel-gson/src/main/docs/json-gson-dataformat.adoc     |  3 ++-
 .../src/main/docs/json-jackson-dataformat.adoc             |  3 ++-
 .../src/main/docs/json-johnzon-dataformat.adoc             |  3 ++-
 .../src/main/docs/json-xstream-dataformat.adoc             |  3 ++-
 8 files changed, 64 insertions(+), 4 deletions(-)


# C) Total Directories Affected

In [149]:
total_directories_affected = repo.git.show("--dirstat",fixing_commit).splitlines()
total_directories_affected = total_directories_affected[6:]
print("Directories Affected:")
for lines in total_directories_affected:
    print(lines)

Directories Affected:
  14.6% components-starter/camel-gson-starter/src/main/java/org/apache/camel/component/gson/springboot/
  14.6% components-starter/camel-jackson-starter/src/main/java/org/apache/camel/component/jackson/springboot/
  14.6% components-starter/camel-johnzon-starter/src/main/java/org/apache/camel/component/johnzon/springboot/
  14.6% components-starter/camel-xstream-starter/src/main/java/org/apache/camel/dataformat/xstream/springboot/
  10.3% components/camel-gson/src/main/docs/
  10.3% components/camel-jackson/src/main/docs/
  10.3% components/camel-johnzon/src/main/docs/
  10.3% components/camel-xstream/src/main/docs/


# D) Total lines of code (including comments and blank lines) Deleted

In [321]:
lines_del_inc_comm = repo.git.show("--shortstat",fixing_commit).splitlines()
lines_del_inc_comm.reverse()
lines_del_inc_comm = lines_del_inc_comm[0].split(',')
lines_del_inc_comm = lines_del_inc_comm[2].split(" ")
print("Total lines of code deleted(including comments and blank lines): "+lines_del_inc_comm[1])

Total lines of code deleted(including comments and blank lines): 4


# E) Total lines of code (including comments and blank lines) Added

In [325]:
lines_add_inc_comm = repo.git.show("--shortstat",fixing_commit).splitlines()
lines_add_inc_comm.reverse()
lines_add_inc_comm = lines_add_inc_comm[0].split(',')
lines_add_inc_comm = lines_add_inc_comm[1].split(" ")
print("Total lines of code Added(including comments and blank lines): "+lines_add_inc_comm[1])

Total lines of code Added(including comments and blank lines): 64


# F) Total lines of code (excluding comments and blank lines) Deleted

In [329]:
lines_del_exc_comm = repo.git.show("-U0","--pretty=""",fixing_commit).splitlines()
lines_added = []
tot = 0
for line in lines_del_exc_comm:
    if re.search("^\-",line):
        l = line.split(" ")
        if(len(l)>1):
            if(len(l[0]) == 1):
                if(l[1][:1] != '*' or l[1][:1] != '/'):
                    tot += 1
print("Total line of code(excluding comments and blank lines) Deleted:" + str(tot))

Total line of code(excluding comments and blank lines) Deleted:0


# G) Total lines of code (excluding comments and blank lines) Added

In [319]:
lines_add_exc_comm = repo.git.show("-U0","--pretty=""",fixing_commit).splitlines()
lines_added = []
tot = 0
for line in lines_add_exc_comm:
    if re.search("^\+",line):
        l = line.split(" ")
        if(len(l)>1):
            if(len(l[0]) == 1):
                if(l[1][:1] != '*' or l[1][:1] != '/'):
                    tot += 1
print("Total line of code(excluding comments and blank lines) Added:" + str(tot))


Total line of code(excluding comments and blank lines) Added:48


# H) Days between the current fixing commit and the previous commit

In [302]:
days = repo.git.log("--format=%cd",fixing_commit,"--",affected_file,"-2").splitlines()
last_commit_date = datetime.strptime(days[0],"%a %b %d %X %Y %z")
last_prev_commit_date = datetime.strptime(days[1],"%a %b %d %X %Y %z")
date_diff = (abs(last_commit_date-last_prev_commit_date))
print("No of days:" + str(date_diff))

No of days:13 days, 0:39:56


# I) No of times each affected file of the current fixing commit been modified in the past since their creation

In [309]:
no_of_times = repo.git.log("--follow","--format=%h",fixing_commit,"--",affected_file).splitlines()
length = len(no_of_times)
print("No of times each affected file of the current fixing commit been modified in the past since their creation:"+ str(length))

No of times each affected file of the current fixing commit been modified in the past since their creation:6


# J) Developers who has modified each affected file since its creation

In [310]:
no_of_times = repo.git.log("--format=%an",fixing_commit,"--",affected_file).splitlines()
s = set(no_of_times)
print("List of Developers:")
for line in s:
    print(line)

List of Developers:
Nicola Ferraro
Aurelien Pupier
Andrea Cosentino
Claus Ibsen


# K) Commits of each Developer

In [314]:
commit_authors = repo.git.shortlog("-sn", "--all").splitlines()
author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    for authors in s:
        if(authors==author):
            author_commits.append((int(commit), authors))

In [315]:
df = pd.DataFrame(author_commits, columns=['Commit','Author'])

In [316]:
df.head()

,Commit,Author
0,19653,Claus Ibsen
1,7758,Andrea Cosentino
2,270,Nicola Ferraro
3,19,Aurelien Pupier
